# Anomaly Detection for Solar Boat Telemetry

This notebook demonstrates how to detect anomalies in telemetry data:
- Battery voltage issues
- Motor current spikes
- GPS signal problems
- Sensor failures

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from solar_regatta import generate_sample_vesc_data, calculate_speeds
from solar_regatta.ml import (
    SimpleAnomalyDetector,
    detect_voltage_anomalies,
    detect_current_spikes,
    detect_gps_anomalies,
)

print("✓ Imports successful!")

## 1. Generate Test Data with Anomalies

In [ ]:
# Generate base telemetry
gps_points, timestamps, speeds_raw, battery_voltage, motor_current = \
    generate_sample_vesc_data(duration_seconds=300, interval=1)

speeds = calculate_speeds(gps_points, timestamps)

# Inject some anomalies for demonstration
battery_voltage = np.array(battery_voltage)
motor_current = np.array(motor_current)
speeds = np.array(speeds)

# Add voltage drop
battery_voltage[100:105] = 10.0  # Sudden voltage drop

# Add current spike
motor_current[150] = 20.0  # Abnormal current spike

# Add impossible speed
speeds[200] = 25.0  # Way too fast

print(f"Generated {len(speeds)} telemetry samples with injected anomalies")

## 2. Detect Voltage Anomalies

In [ ]:
# Detect voltage issues
low_voltage, high_voltage, rapid_change = detect_voltage_anomalies(
    battery_voltage,
    low_threshold=10.5,
    high_threshold=14.0,
    rate_threshold=0.5
)

print(f"Voltage Anomalies Detected:")
print(f"  Low voltage events: {low_voltage.sum()}")
print(f"  High voltage events: {high_voltage.sum()}")
print(f"  Rapid changes: {rapid_change.sum()}")

# Visualize
plt.figure(figsize=(15, 5))
plt.plot(battery_voltage, 'b-', label='Voltage', linewidth=2)
plt.axhline(y=10.5, color='r', linestyle='--', label='Low Threshold')
plt.axhline(y=14.0, color='orange', linestyle='--', label='High Threshold')

# Mark anomalies
plt.scatter(np.where(low_voltage)[0], battery_voltage[low_voltage], 
           color='red', s=100, marker='x', label='Low Voltage', zorder=5)
plt.scatter(np.where(high_voltage)[0], battery_voltage[high_voltage], 
           color='orange', s=100, marker='x', label='High Voltage', zorder=5)
plt.scatter(np.where(rapid_change)[0], battery_voltage[rapid_change], 
           color='purple', s=100, marker='o', label='Rapid Change', zorder=5)

plt.xlabel('Time Step')
plt.ylabel('Voltage (V)')
plt.title('Battery Voltage Anomaly Detection', fontweight='bold', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Detect Current Spikes

In [ ]:
# Detect current anomalies
current_spikes = detect_current_spikes(
    motor_current,
    spike_threshold=3.0,
    window_size=5
)

print(f"Current Spikes Detected: {current_spikes.sum()}")
print(f"Spike locations: {np.where(current_spikes)[0].tolist()}")

# Visualize
plt.figure(figsize=(15, 5))
plt.plot(motor_current, 'g-', label='Current', linewidth=2)
plt.scatter(np.where(current_spikes)[0], motor_current[current_spikes], 
           color='red', s=200, marker='*', label='Spike Detected', zorder=5)

plt.xlabel('Time Step')
plt.ylabel('Current (A)')
plt.title('Motor Current Spike Detection', fontweight='bold', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Detect GPS Anomalies

In [ ]:
# Detect GPS issues
impossible_speed, impossible_accel = detect_gps_anomalies(
    speeds,
    max_speed=15.0,
    max_acceleration=2.0
)

print(f"GPS Anomalies Detected:")
print(f"  Impossible speeds: {impossible_speed.sum()}")
print(f"  Impossible accelerations: {impossible_accel.sum()}")

# Visualize
plt.figure(figsize=(15, 5))
plt.plot(speeds, 'b-', label='Speed', linewidth=2)
plt.axhline(y=15.0, color='r', linestyle='--', label='Max Speed Threshold')

plt.scatter(np.where(impossible_speed)[0], speeds[impossible_speed], 
           color='red', s=200, marker='x', label='Impossible Speed', zorder=5)
plt.scatter(np.where(impossible_accel)[0], speeds[impossible_accel], 
           color='orange', s=100, marker='o', label='Impossible Accel', zorder=5)

plt.xlabel('Time Step')
plt.ylabel('Speed (m/s)')
plt.title('GPS Anomaly Detection', fontweight='bold', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Statistical Anomaly Detection

In [ ]:
# Use Z-score method for general anomaly detection
detector = SimpleAnomalyDetector(method='zscore', threshold=3.0)

# Train on first half (assumed normal)
detector.fit(speeds[:len(speeds)//2])

# Detect on full data
anomalies = detector.predict(speeds)

print(f"Statistical Anomalies: {anomalies.sum()}")
print(f"Anomaly rate: {100 * anomalies.sum() / len(speeds):.2f}%")

# Visualize
plt.figure(figsize=(15, 5))
plt.plot(speeds, 'b-', alpha=0.7, label='Speed')
plt.scatter(np.where(anomalies)[0], speeds[anomalies], 
           color='red', s=100, marker='x', label=f'Anomalies ({anomalies.sum()})', zorder=5)

plt.xlabel('Time Step')
plt.ylabel('Speed (m/s)')
plt.title('Statistical Anomaly Detection (Z-score)', fontweight='bold', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Combined Anomaly Report

In [ ]:
# Combine all anomalies
all_anomalies = (
    low_voltage[:len(speeds)] | 
    high_voltage[:len(speeds)] | 
    rapid_change[:len(speeds)] | 
    current_spikes[:len(speeds)] | 
    impossible_speed | 
    impossible_accel | 
    anomalies
)

print("\n" + "="*70)
print("ANOMALY DETECTION SUMMARY")
print("="*70)
print(f"Total samples analyzed: {len(speeds)}")
print(f"Total anomalies found: {all_anomalies.sum()}")
print(f"Anomaly rate: {100 * all_anomalies.sum() / len(speeds):.2f}%")
print("\nBreakdown by type:")
print(f"  Low voltage:          {low_voltage[:len(speeds)].sum():3d}")
print(f"  High voltage:         {high_voltage[:len(speeds)].sum():3d}")
print(f"  Rapid voltage change: {rapid_change[:len(speeds)].sum():3d}")
print(f"  Current spikes:       {current_spikes[:len(speeds)].sum():3d}")
print(f"  Impossible speed:     {impossible_speed.sum():3d}")
print(f"  Impossible accel:     {impossible_accel.sum():3d}")
print(f"  Statistical outliers: {anomalies.sum():3d}")
print("="*70)

# Comprehensive visualization
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Speed
axes[0].plot(speeds, 'b-', alpha=0.7)
axes[0].scatter(np.where(all_anomalies)[0], speeds[all_anomalies], 
               color='red', s=50, marker='x', zorder=5)
axes[0].set_ylabel('Speed (m/s)')
axes[0].set_title('Speed with Detected Anomalies', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Voltage
axes[1].plot(battery_voltage[:len(speeds)], 'g-', alpha=0.7)
axes[1].scatter(np.where(all_anomalies)[0], battery_voltage[:len(speeds)][all_anomalies], 
               color='red', s=50, marker='x', zorder=5)
axes[1].set_ylabel('Voltage (V)')
axes[1].set_title('Voltage with Detected Anomalies', fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Current
axes[2].plot(motor_current[:len(speeds)], 'orange', alpha=0.7)
axes[2].scatter(np.where(all_anomalies)[0], motor_current[:len(speeds)][all_anomalies], 
               color='red', s=50, marker='x', zorder=5)
axes[2].set_ylabel('Current (A)')
axes[2].set_xlabel('Time Step')
axes[2].set_title('Current with Detected Anomalies', fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated various anomaly detection techniques:

✅ **Voltage Monitoring** - Detect low/high voltage and rapid changes  
✅ **Current Spike Detection** - Identify abnormal motor behavior  
✅ **GPS Validation** - Flag impossible speeds and accelerations  
✅ **Statistical Methods** - Z-score and IQR for general outliers  

### Use Cases:
- **Pre-race checks**: Validate sensor data quality
- **Real-time monitoring**: Alert on critical issues during race
- **Post-race analysis**: Identify equipment problems
- **Data cleaning**: Filter bad data before ML training

### Next Steps:
1. Integrate with real VESC data stream
2. Add real-time alerting
3. Create dashboard for monitoring
4. Log anomalies for maintenance tracking